# **Crypto On-chain Data Analysis**
This notebook analyzes historical tokenomics for major cryptocurrencies (BTC, ETH, BNB, SOL) using data from CoinGecko API.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pycoingecko import CoinGeckoAPI

# Initialize CoinGecko API client
cg = CoinGeckoAPI()

# Set default template for plotly
import plotly.io as pio
pio.templates.default = "plotly_dark"


In [2]:
# Define the cryptocurrencies we want to analyze
crypto = pd.DataFrame({
    'symbol': ['BTC', 'ETH', 'BNB', 'SOL'],
    'name': ['Bitcoin', 'Ethereum', 'Binance Coin', 'Solana'],
    'coingecko_id': ['bitcoin', 'ethereum', 'binancecoin', 'solana']
})

print("Cryptocurrencies to analyze:")
print(crypto)

Cryptocurrencies to analyze:
  symbol          name coingecko_id
0    BTC       Bitcoin      bitcoin
1    ETH      Ethereum     ethereum
2    BNB  Binance Coin  binancecoin
3    SOL        Solana       solana


In [ ]:
# Fetching Tokenomics Data
tokenomics_data = []

for _, row in crypto.iterrows():
    try:
        print(f"Fetching data for {row['name']}...")
        # Fetch detailed coin data from CoinGecko
        coin_data = cg.get_coin_by_id(row['coingecko_id'])
        
        data = {
            'symbol': row['symbol'],
            'current_price': coin_data['market_data']['current_price']['usd'],
            'market_cap': coin_data['market_data']['market_cap']['usd'],
            'total_volume': coin_data['market_data']['total_volume']['usd'],
            'circulating_supply': coin_data['market_data']['circulating_supply'],
            'total_supply': coin_data['market_data']['total_supply'],
            'max_supply': coin_data['market_data']['max_supply'],
            'fdv': coin_data['market_data'].get('fully_diluted_valuation', {}).get('usd'),
        }
        tokenomics_data.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {row['symbol']}: {e}")

# Convert to DataFrame
tokenomics_df = pd.DataFrame(tokenomics_data)



print("\nTokenomics Data:")
display(tokenomics_df)

Fetching data for Bitcoin...
Fetching data for Ethereum...
Fetching data for Binance Coin...
Fetching data for Ethereum...
Fetching data for Binance Coin...
Fetching data for Solana...

Tokenomics Data:
Fetching data for Solana...

Tokenomics Data:


,symbol,current_price,market_cap,total_volume,circulating_supply,total_supply,max_supply,fdv
0,BTC,"$94,213.00","$1,870,400,825,262.00","$30,428,952,301.00","19,857,331","19,857,331","21,000,000","$1,870,400,825,262.00"
1,ETH,"$1,777.06","$214,299,396,386.00","$14,804,568,870.00","120,725,291","120,725,291",N/A,"$214,299,396,386.00"
2,BNB,$602.63,"$87,912,802,734.00","$661,772,980.00","145,887,576","145,887,576","200,000,000","$87,912,802,734.00"
3,SOL,$146.76,"$75,969,072,889.00","$4,236,990,640.00","517,548,302","599,468,745",N/A,"$87,993,882,990.00"
